# Modelos Predictivos Aplicados

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import emoji

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('spanish'))

import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juanj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv('db.csv')
df['date'] = pd.to_datetime(df['date'])
df["Palabras"] = [len(msg.split()) for msg in df["tweet"]]
df.drop("time", axis = 1, inplace = True)
df

,date,tweet,Palabras
0,2023-01-01,"Resuelto, muchísimas gracias , excelente servi...",8
1,2023-01-02,"Muchas gracias, espero su dm",5
2,2023-01-02,Muchas gracias!,2
3,2023-01-02,Algo similar me paso. Quería renovar mi token ...,28
4,2023-01-02,Yeeeei! a través de mi cuenta en acabo de cont...,47
...,...,...,...
807,2024-04-21,"Ahora confirmo, gracias estimado",4
808,2024-04-23,😔 de acuerdo!! Muchas gracias.,5
809,2024-04-23,"Tío cuando me van a graduar, tengo la TDC gara...",32
810,2024-04-25,"una duda, con quién tengo que ver si tengo pro...",38


### Desemojizar

In [4]:
for i in range(len(df)):
    df.loc[i,'tweet'] = emoji.demojize(df.loc[i,'tweet'], language='es')
df

,date,tweet,Palabras
0,2023-01-01,"Resuelto, muchísimas gracias , excelente servi...",8
1,2023-01-02,"Muchas gracias, espero su dm",5
2,2023-01-02,Muchas gracias!,2
3,2023-01-02,Algo similar me paso. Quería renovar mi token ...,28
4,2023-01-02,Yeeeei! a través de mi cuenta en acabo de cont...,47
...,...,...,...
807,2024-04-21,"Ahora confirmo, gracias estimado",4
808,2024-04-23,:cara_desanimada: de acuerdo!! Muchas gracias.,5
809,2024-04-23,"Tío cuando me van a graduar, tengo la TDC gara...",32
810,2024-04-25,"una duda, con quién tengo que ver si tengo pro...",38


### Preprocesamiento de Datos

In [5]:
# Función para preprocesar el texto
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('spanish')]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    
    
    # ----------- LIMPIAR CARACTERES Y PALABRAS ----------------
    processed_text = processed_text.replace(',', '')
    processed_text = processed_text.replace('.', '')
    processed_text = processed_text.replace(':', '')
    processed_text = processed_text.replace(',', '')
    processed_text = processed_text.replace('!', '')
    processed_text = processed_text.replace('¡', '')
    #processed_text = processed_text.replace('gracias', '')
    #processed_text = processed_text.replace('muchas', '')
    
    #processed_text = processed_text.replace('_', ' ')
    #processed_text = processed_text.replace('cara', '')
    

    return processed_text

In [6]:
df['procTweet'] = df['tweet'].apply(preprocess_text)
# Eliminar las filas con texto vacío
df = df.loc[df['procTweet'] != '', :]
df

,date,tweet,Palabras,procTweet
0,2023-01-01,"Resuelto, muchísimas gracias , excelente servi...",8,resuelto muchísimas gracias excelente servic...
1,2023-01-02,"Muchas gracias, espero su dm",5,muchas gracias espero dm
2,2023-01-02,Muchas gracias!,2,muchas gracias
3,2023-01-02,Algo similar me paso. Quería renovar mi token ...,28,similar paso quería renovar token poder hacer...
4,2023-01-02,Yeeeei! a través de mi cuenta en acabo de cont...,47,yeeeei través cuenta acabo contratar cuenta a...
...,...,...,...,...
807,2024-04-21,"Ahora confirmo, gracias estimado",4,ahora confirmo gracias estimado
808,2024-04-23,:cara_desanimada: de acuerdo!! Muchas gracias.,5,cara_desanimada acuerdo muchas gracias
809,2024-04-23,"Tío cuando me van a graduar, tengo la TDC gara...",32,tío van graduar tdc garantizada hace año saqu...
810,2024-04-25,"una duda, con quién tengo que ver si tengo pro...",38,duda quién ver si problemas aumentar línea cr...


## Predecir Sentimiento

In [7]:
from joblib import load
modelSentiment = load('modelSentiment.joblib')
scaler = load('scaler.joblib')
vectorizer = load('vectorizer.joblib')

In [8]:
corpus = []
stemmer = PorterStemmer()
for i in range(0, df.shape[0]):
  review = re.sub('[^a-zA-Z]', ' ', df.iloc[i]['procTweet'])
  review = review.lower().split()
  review = [stemmer.stem(word) for word in review if not word in STOPWORDS]
  review = ' '.join(review)
  corpus.append(review)

In [9]:
#Storing independent and dependent variables in X and y
X = vectorizer.transform(corpus).toarray()

In [10]:
X = scaler.transform(X)

In [11]:
df['Sentimiento'] = modelSentiment.predict(X) 
df

C:\Users\juanj\AppData\Local\Temp\ipykernel_9876\3611637185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentimiento'] = modelSentiment.predict(X)


,date,tweet,Palabras,procTweet,Sentimiento
0,2023-01-01,"Resuelto, muchísimas gracias , excelente servi...",8,resuelto muchísimas gracias excelente servic...,1.0
1,2023-01-02,"Muchas gracias, espero su dm",5,muchas gracias espero dm,1.0
2,2023-01-02,Muchas gracias!,2,muchas gracias,1.0
3,2023-01-02,Algo similar me paso. Quería renovar mi token ...,28,similar paso quería renovar token poder hacer...,0.0
4,2023-01-02,Yeeeei! a través de mi cuenta en acabo de cont...,47,yeeeei través cuenta acabo contratar cuenta a...,0.0
...,...,...,...,...,...
807,2024-04-21,"Ahora confirmo, gracias estimado",4,ahora confirmo gracias estimado,1.0
808,2024-04-23,:cara_desanimada: de acuerdo!! Muchas gracias.,5,cara_desanimada acuerdo muchas gracias,1.0
809,2024-04-23,"Tío cuando me van a graduar, tengo la TDC gara...",32,tío van graduar tdc garantizada hace año saqu...,1.0
810,2024-04-25,"una duda, con quién tengo que ver si tengo pro...",38,duda quién ver si problemas aumentar línea cr...,0.0


## Predecir Relevancia

In [12]:
from joblib import load
modelRel = load('modelRel.joblib')

In [13]:
df['relevance'] = modelRel.predict(X) 
df

C:\Users\juanj\AppData\Local\Temp\ipykernel_9876\3846445833.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['relevance'] = modelRel.predict(X)


,date,tweet,Palabras,procTweet,Sentimiento,relevance
0,2023-01-01,"Resuelto, muchísimas gracias , excelente servi...",8,resuelto muchísimas gracias excelente servic...,1.0,1
1,2023-01-02,"Muchas gracias, espero su dm",5,muchas gracias espero dm,1.0,0
2,2023-01-02,Muchas gracias!,2,muchas gracias,1.0,0
3,2023-01-02,Algo similar me paso. Quería renovar mi token ...,28,similar paso quería renovar token poder hacer...,0.0,1
4,2023-01-02,Yeeeei! a través de mi cuenta en acabo de cont...,47,yeeeei través cuenta acabo contratar cuenta a...,0.0,1
...,...,...,...,...,...,...
807,2024-04-21,"Ahora confirmo, gracias estimado",4,ahora confirmo gracias estimado,1.0,0
808,2024-04-23,:cara_desanimada: de acuerdo!! Muchas gracias.,5,cara_desanimada acuerdo muchas gracias,1.0,0
809,2024-04-23,"Tío cuando me van a graduar, tengo la TDC gara...",32,tío van graduar tdc garantizada hace año saqu...,1.0,1
810,2024-04-25,"una duda, con quién tengo que ver si tengo pro...",38,duda quién ver si problemas aumentar línea cr...,0.0,1


## Predecir Topic

In [14]:
from joblib import load
modelTopic = load('modelTopic.joblib')

In [15]:
df['topic'] = modelTopic.predict(X) 
df

C:\Users\juanj\AppData\Local\Temp\ipykernel_9876\2032292472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topic'] = modelTopic.predict(X)


,date,tweet,Palabras,procTweet,Sentimiento,relevance,topic
0,2023-01-01,"Resuelto, muchísimas gracias , excelente servi...",8,resuelto muchísimas gracias excelente servic...,1.0,1,misc
1,2023-01-02,"Muchas gracias, espero su dm",5,muchas gracias espero dm,1.0,0,misc
2,2023-01-02,Muchas gracias!,2,muchas gracias,1.0,0,misc
3,2023-01-02,Algo similar me paso. Quería renovar mi token ...,28,similar paso quería renovar token poder hacer...,0.0,1,misc
4,2023-01-02,Yeeeei! a través de mi cuenta en acabo de cont...,47,yeeeei través cuenta acabo contratar cuenta a...,0.0,1,misc
...,...,...,...,...,...,...,...
807,2024-04-21,"Ahora confirmo, gracias estimado",4,ahora confirmo gracias estimado,1.0,0,misc
808,2024-04-23,:cara_desanimada: de acuerdo!! Muchas gracias.,5,cara_desanimada acuerdo muchas gracias,1.0,0,misc
809,2024-04-23,"Tío cuando me van a graduar, tengo la TDC gara...",32,tío van graduar tdc garantizada hace año saqu...,1.0,1,support
810,2024-04-25,"una duda, con quién tengo que ver si tengo pro...",38,duda quién ver si problemas aumentar línea cr...,0.0,1,misc


In [16]:
df.to_csv('dbParaAnalisis.csv', index = False)

## Pruebas en "Tiempo Real"

In [43]:
def clasif(msgs):
    resultados = []
    for msg in msgs:
        orig = msg
        corpus = []
        stemmer = PorterStemmer()
        review = re.sub('[^a-zA-Z]', ' ', msg)
        review = review.lower().split()
        review = [stemmer.stem(word) for word in review if not word in STOPWORDS]
        review = ' '.join(review)
        corpus.append(review)
        
        msg = vectorizer.transform(corpus).toarray()
        msg = scaler.transform(msg)
        resultado = {'Mensaje': orig,
                     'Sentimiento': modelSentiment.predict(msg)[0], 
                     'Relevancia': modelRel.predict(msg)[0], 
                     'Topico': modelTopic.predict(msg)[0]}
        resultados.append(resultado)
        
    return resultados

### Casos de Prueba Chidos

In [44]:
ans = clasif(['Son una basura. Ahora cada dia hay intermitencia. Estoy igual ayer y hoy igual sin poder hacer uso de nuestro dinero. Cambio de banco total'])
ans

[{'Mensaje': 'Son una basura. Ahora cada dia hay intermitencia. Estoy igual ayer y hoy igual sin poder hacer uso de nuestro dinero. Cambio de banco total',
  'Sentimiento': 0.0,
  'Relevancia': 1,
  'Topico': 'misc'}]

In [45]:
pruebas = pd.read_csv('dataTest.csv')
pruebas = list(pruebas['tweet'])

In [46]:
clasif(pruebas)

[{'Mensaje': 'Hola @heybanco\n su "personal" del hospitality corriendo gente y diciéndoles que solo tienen 15 min para estar ahí, buena idea, mala ejecución y más con personal de seguridad en mal plan! Se inició bien y se terminó mal! #akamba 2do vid vieron que los grabé y cambiaron el speech',
  'Sentimiento': 1.0,
  'Relevancia': 1,
  'Topico': 'misc'},
 {'Mensaje': '¿ @heybanco\n por qué “debo” recompensas?',
  'Sentimiento': 1.0,
  'Relevancia': 0,
  'Topico': 'misc'},
 {'Mensaje': 'No se que esta mas gato, que @heybanco\n mande estos mensajes aun faltando pra el pago, o yo por tener su pinchi cuenta pitera. Yo veda?',
  'Sentimiento': 1.0,
  'Relevancia': 1,
  'Topico': 'misc'},
 {'Mensaje': '@CondusefMX\n & @Profeco\n, hola, lamentablemete, ya por la cuarta vez, he tenido muchos problemas con @heybanco\n con depositos, transferencias, & reembolsos. Esto ya con clabe de rastreo & mas comprobantes. #HeyBanco se arrastra, miente, culpa a tod@s,sale con mil excusas! Es un gran',
  'S